In [1]:
import pandas as pd

In [2]:
yelpframe = pd.read_csv('./data/all_reviews_with_businesses_no_text.csv', index_col=[0])


In [3]:
print(yelpframe.head())

review_id                 user_id             business_id  \
0  xQY8N_XvtGbearJ5X4QryQ  OwjRMXRC0KyPrIlcjaXeFQ  -MhfebM0QIsKt87iDN-FNw   
1  t7xOZF5UKXjSpVcXLOSAgw  owbC7FP8SNAlwv6f9S5Stw  -MhfebM0QIsKt87iDN-FNw   
2  MimB5Xh85rG7phUMPrShag  v9vGnjphb0Hta0lvtf5haA  -MhfebM0QIsKt87iDN-FNw   
3  sLkT7J06L4TK4PiRUFax2g  AXuHgGQoNPkiSXTxHlQc0A  -MhfebM0QIsKt87iDN-FNw   
4  cnV5xtm6WuyaLfot9uWbDg  LkWNo83Lg92C5V4JEyxOZA  -MhfebM0QIsKt87iDN-FNw   

   stars                 date                          name       city state  \
0      2  2015-04-15 05:21:16  Bellagio Gallery of Fine Art  Las Vegas    NV   
1      2  2014-03-14 08:24:25  Bellagio Gallery of Fine Art  Las Vegas    NV   
2      3  2015-10-07 22:16:59  Bellagio Gallery of Fine Art  Las Vegas    NV   
3      2  2015-11-18 22:20:55  Bellagio Gallery of Fine Art  Las Vegas    NV   
4      3  2010-10-10 01:27:31  Bellagio Gallery of Fine Art  Las Vegas    NV   

                                          categories  
0  Shopping, Arts

In [4]:
yelpframe.shape

(8021122, 9)

In [6]:
phxframe = yelpframe[yelpframe['state'] == 'AZ'].copy()

In [7]:
phxframe.shape

(2504395, 9)

In [8]:
phxframe['city'] = phxframe['city'].apply(lambda x: str(x).lower())
phxframe['city'] = phxframe['city'].apply(lambda x: x.strip(' '))

In [9]:
# filtering cities down to only those with 1000 reviews or more
citycounts = phxframe['city'].value_counts()
citiestokeep = citycounts[(citycounts >= 1000)].index.tolist()

In [10]:
phxframe = phxframe[(phxframe['city'].isin(citiestokeep))]

In [11]:
# append the user id and business id together for each review - this id will be unique to a user-business pair
phxframe['repeat_review_id'] = phxframe['user_id']+phxframe['business_id']

In [12]:
# look for dupes
phxframe['repeat_review_id'].value_counts()

JlQ-9fc61X9lbzZN6ZjORQJ3H6VSIgUTlACkb_HPFA8w    62
4d13xAX2jp2EbGF8I9eZZwK_-l_AhoGQvYDeGNgm52qQ    36
GCDf7UWiDfyg2JenNuYgZQQ3SAIIFURXZu421zG17N0A    28
BZVV3Q8KSSOyJBidrT-cNAY-hmtitiRfg7vmc3N-lTww    25
M3YpU8fRyTZAQIn08bDT4gVeRHiEbUe_vHprYn8mnezA    25
                                                ..
f6XNrxk8z90XMD6v1wKOqQmUSbQH2LePuJEygPlHn9Jg     1
KGoBYyDOy1KWgHe_p1CTygh6Y1-KfN2XBkcrV85CS8yg     1
OfF99HshLdOeW5haaUgRfgfXUJ9d5R6kNXrnAouByB6g     1
zWp6r0OTdS0VNjJW7ZmEMg5nZVVPO_cYH9aQeClTP9zA     1
V_5qEY4-1YzO_vI88Tx5WQkHpXJ2vciumu2KhNHg0hYA     1
Name: repeat_review_id, Length: 2398712, dtype: int64

In [13]:
phxframe['date'] = pd.to_datetime(phxframe['date'])

In [14]:
# for duplicate user-business pairs, keep only the most recent review.
phxframe_no_dupes = phxframe.sort_values(by='date', ascending=False).drop_duplicates(subset=['repeat_review_id'],keep='first').copy()

In [15]:
phxframe.shape

(2499124, 10)

In [16]:
phxframe_no_dupes.to_csv('./data/all_reviews_no_dupes.csv')

In [ ]:
import sqlite3

In [17]:
businessframe = pd.read_json('../labs_data/yelp_academic_dataset_business.json', lines=True, orient='records')

In [18]:
businessframe.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [19]:
# filtering businesses by the same criteria as reviews
phxbizframe = businessframe[businessframe['state'] == 'AZ'].copy()

In [20]:
phxbizframe['city'] = phxbizframe['city'].apply(lambda x: str(x).lower())
phxbizframe['city'] = phxbizframe['city'].apply(lambda x: x.strip(' '))
phxbizframe = phxbizframe[(phxbizframe['city'].isin(citiestokeep))]

In [21]:
# convert the category string to a list
phxbizframe['cat_list'] = phxbizframe['categories'].apply(lambda x: str(x).replace(' ', '').split(','))

In [22]:
# convert the list to one-hots
from sklearn.preprocessing import MultiLabelBinarizer


# Binarise labels
mlb = MultiLabelBinarizer()
expanded_cat_data = mlb.fit_transform(phxbizframe['cat_list'])
cat_classes = ['category_' + x for x in mlb.classes_]


expanded_cats = pd.DataFrame(expanded_cat_data)
expanded_cats.columns = cat_classes

In [23]:
expanded_cats.head()

,category_&Probates,category_3DPrinting,category_ATVRentals/Tours,category_AcaiBowls,category_Accessories,category_Accountants,category_AcneTreatment,category_ActiveLife,category_Acupuncture,category_AddictionMedicine,...,category_WineTastingRoom,category_WineTours,category_Wineries,category_Women'sClothing,category_WorkersCompensationLaw,category_Wraps,category_YelpEvents,category_Yoga,category_Ziplining,category_Zoos
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# if the category doesn't apply to at least 1000 businesses, drop it.
catcounts = expanded_cats.sum().sort_values(ascending=False)
catstodrop = catcounts[(catcounts < 1000)].index.tolist()

In [25]:
expanded_cats.drop(columns=catstodrop, inplace=True)

In [26]:
expanded_cats.shape

(60540, 42)

In [27]:
phxbizframe.reset_index(drop=True, inplace=True)

In [28]:
concatenated = pd.concat([phxbizframe, expanded_cats], axis=1)

In [29]:
# drop unused features
concatenated.drop(columns=['postal_code','latitude','longitude','is_open','attributes','cat_list','hours'], inplace=True)
concatenated.rename(columns={'stars':'aggregate_rating'}, inplace=True)

In [30]:
concatenated.head()

,business_id,name,address,city,state,aggregate_rating,review_count,categories,category_ActiveLife,category_American(New),...,category_NailSalons,category_Nightlife,category_Pets,category_Pizza,category_ProfessionalServices,category_RealEstate,category_Restaurants,category_Sandwiches,category_Shopping,category_SkinCare
0,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",scottsdale,AZ,5.0,4,"Health & Medical, Fitness & Instruction, Yoga,...",1,0,...,0,0,0,0,0,0,0,0,0,0
1,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,mesa,AZ,4.5,26,"Home Services, Plumbing, Electricians, Handyma...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",gilbert,AZ,4.5,38,"Auto Repair, Automotive, Oil Change Stations, ...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,6910 E Southern Ave,mesa,AZ,5.0,18,"Auto Repair, Oil Change Stations, Automotive, ...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,JjJs3o60uQCfctDjs45cmA,Convertabath,"116 N Roosevelt Ave, Bldg B, Ste 124",chandler,AZ,2.5,10,"Contractors, Home Services, Local Services",0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
conn = sqlite3.connect('../labs-yelp-api/lightfm/db_stuff/business_data.sqlite3')
concatenated.to_sql(name='business_data', con=conn, if_exists='replace')

In [34]:
conn.close()